# A Framework for Verification of BTs using LTL: Execution Example

O. Biggar and M. Zamani

Here we present the execution of the code for the example presented in A Framework for Formal Verification of Behavior Trees using Linear Temporal Logic. The implementation here is basic and not optimised---the point of this example is to show clearly that the ideas presented in the paper can be easily transferred into a working implementation. The detailed discussion of the algorithm and the ideas behind the example are omitted here because they are presented in the paper itself.

In [1]:
from bt_verification import Behaviour, combine, entails, preconditions
import spot
spot.setup()

Here we create some behaviors, as presented in Table 1. The arguments of this constructor are: name, success conditions, failure conditions, guarantee. We use Spot's LTL syntax: `G` globally (always), `F` eventually, `X` next, `!` Boolean not, `&` Boolean and, `|` Boolean or and `->` as implication.

In [2]:
send = Behaviour('send','false','!data','F(sent & !data)')
panels = Behaviour('panels','false','false','charge & (day -> F!lowpower)')
hibernate = Behaviour('hibernate','false','false','hibernating')
getdata = Behaviour('getData','data','false','Fdata')

We also construct a behavior for the environment, `E`.

In [3]:
E = Behaviour('env','false','false',"""GFday & FG!storm & (FGlowpower | FG!lowpower)
              & (dead -> (lowpower & !(charge | hibernating))) & (damaged -> (storm & !hibernating))""")

Now we use the operators defined in the paper to construct a behavior modelling the entire tree `T`, which we now call `B`. For this we use the `combine` function, which takes a string giving the structure of the tree and a list of behaviors contained in the tree. Behaviors modelling conditions are implicitly created where necessary.

In [4]:
B = combine('(lowpower->panels)?(storm->hibernate)?(getData->send)',
            [panels,hibernate,getdata,send])

Here we construct formulas to give the sets of logical runs of `B` and `E`. Also, we construct the formula representing the desired specification.

In [5]:
log_run_B = spot.formula.G(spot.formula.Or((B.success,B.failure,B.guarantee)))
log_run_E = spot.formula.G(spot.formula.Or((E.success,E.failure,E.guarantee)))
specification = spot.formula('G!damaged & G!dead & Fsent')

Now we need only check that `log_run_B & log_run_E` entails `specification`, which is done with the following function `entails`.

In [27]:
entails(spot.formula.And((log_run_B,log_run_E)),specification)

Not satisfied. One counterexample is
Prefix:
  0,2
  |  charge & damaged & !dead & !hibernating & lowpower & storm
  0,1
  |  !damaged & data & !dead & !lowpower & !storm
Cycle:
  1,1
  |  !damaged & data & !day & !dead & !lowpower & !storm	{0}
  1,1
  |  !damaged & !data & day & !dead & !lowpower & !sent & !storm	{2}
  1,1
  |  !damaged & !data & !day & !dead & !lowpower & sent & !storm	{1}
  3,1
  |  !damaged & data & !day & !dead & !lowpower & !storm	{0}



False

By observing the generated counterexample, we note that the robot is `damaged` as `lowpower` and `storm` are simultaneously true in the initial state but the robot chooses to charge rather than hibernate. To correct this, we swap the order of these two subtrees and try again:

In [28]:
B = combine('(storm->hibernate)?(lowpower->panels)?(getData->send)',
            [panels,hibernate,getdata,send])

log_run_B = spot.formula.G(spot.formula.Or((B.success,B.failure,B.guarantee)))
log_run_E = spot.formula.G(spot.formula.Or((E.success,E.failure,E.guarantee)))
specification = spot.formula('G!damaged & G!dead & Fsent')

entails(spot.formula.And((log_run_B,log_run_E)),specification)

True

So now this tree is correct. Now suppose we wish to replace the Action GetData with a new subtree. We introduce some new behaviors:

In [9]:
gotobase = Behaviour('goToBase','atBase','false','FatBase & ((holding & Xholding)|(!holding & X!holding))')
place = Behaviour('place','false','!holding','atBase -> F(!holding & sample)')
getrock = Behaviour('getRock','holding','false','Fholding')
analyse = Behaviour('analyse','false','!sample','!broken -> Fdata')
fix = Behaviour('fix','!broken','false','F!broken')

And combine these to form a new behavior `G`:

In [10]:
G = combine('data?(((sample ? (((holding ? getRock) -> (atBase ? goToBase)) -> place)) -> ((!broken) ? fix)) -> analyse)',
            [gotobase,place,getrock,analyse,fix])
print("Success:",G.success)
print("Failure:",G.failure)

Success: data
Failure: 0


We observe that the success and failure conditions are the same as that of `getData`. Now, to check if the modified tree is still correct, we firstly check the safety conditions. To do this, we construct a behavior `idle` which is strongly refined by `G` and check if the safety conditions are still satisfied.

In [11]:
idle = Behaviour('idle','data','false','true')
safety = spot.formula('G!damaged & G!dead')

In [13]:
B_id = combine('(storm->hibernate)?(lowpower->panels)?(idle->send)',
                [panels,hibernate,send,idle])

log_run_B_id = spot.formula.G(spot.formula.Or((B_id.success,B_id.failure,B_id.guarantee)))
log_run_E = spot.formula.G(spot.formula.Or((E.success,E.failure,E.guarantee)))
B_id_and_E = spot.formula.And((log_run_B_id,log_run_E))

entails(B_id_and_E,safety)

True

Now that we know that the safety conditions must hold in any strong refinement of `idle`. As any refinement of `getData` is a strong refinement of `idle`, we need only check whether the liveness condition `F sent` still holds. Firstly, we will prove that `FG prec_K` holds, and then show that `G` refines `getData`.

In [14]:
preconditions('(storm->hibernate)?(lowpower->panels)?(getData->send)',
              'getData',
              [panels,hibernate,send,getdata])

spot.formula("!lowpower & !storm")

In [15]:
entails(B_id_and_E,spot.formula('FG(!lowpower & !storm)'))

True

Then, for any strong refinement of `idle`, we can conclude that `G!dead & G!dead & FG!lowpower & FG!storm` holds. Now to show that the modified tree with `G` works we need only show that `G` refines `getData`, when run against the environment. Many of the variables, such as `day`, `lowpower`, `storm`, etc. cannot affect whether this is true because they do not exist in the new behavior or the new specification `F data`, so we can remove them from the environment. We create a new environment behavior `E2`:

In [34]:
E2 = Behaviour('env','false','false','FG broken | FG!broken')

Now we need to check that this behavior satisfies `G(data | false | F data)`. If this is satisfied, then we can replace `getData` by `G` in the original tree above and it will remain correct, as proved in the paper.

In [35]:
log_run_G = spot.formula.G(spot.formula.Or((G.success,G.failure,G.guarantee)))
log_run_E2 = spot.formula.G(spot.formula.Or((E2.success,E2.failure,E2.guarantee)))

In [36]:
entails(spot.formula.And((log_run_G,log_run_E2)),spot.formula('data | Fdata'))

True

So `G` refines `getData`, as required.